In [4]:
import pandas as pd
import datetime

In [5]:
def calculate_week_of_month(date):
    first_day_of_month = date + pd.to_timedelta(f'-{date.day} days')
    return (date - first_day_of_month).days // 7 + 1

In [6]:
def process_csv(input_file, output_file):
    df = pd.read_csv(input_file)

    df.fillna(method='ffill', inplace=True)

    df['start_date'] = pd.to_datetime(df['Event Start Read Date'])

    df['end_date'] = pd.to_datetime(df['Event Stop Read Date'])

    df['duration_days'] = (df['end_date'] - df['start_date']).dt.days + 1

    # Create a new DataFrame to store the updated data
    new_df = []
    for index, row in df.iterrows():
        start_date = row['start_date']
        end_date = row['end_date']
        days = row['duration_days']
        average = row['Avg Daily Flow (Event)']
        week_of_month = calculate_week_of_month(start_date)

        for day in range(days):
            new_row = {
                'Cycle/Route': row['Cycle/Route'],
                'Address': row['Address'],
                'MIU ID': row['MIU ID'],
                'Meter Number': row['Meter Number'],
                'Meter Size': row['Meter Size'],
                'Event Start Read Date': start_date + pd.to_timedelta(f'{day} days'),
                'Event Stop Read Date': start_date + pd.to_timedelta(f'{day} days'),
                'Days': 1,
                'Avg Daily Flow (Event)': row['Avg Daily Flow (Event)'],
                'Avg Daily Flow (Prior)': row['Avg Daily Flow (Prior)'],
                'UOM': row['UOM'],
                'Last Read Date': row['Last Read Date'],
                'Week of Month': week_of_month,
                'Day of Week': (start_date + pd.to_timedelta(f'{day} days')).strftime('%A')
            }
            new_df.append(new_row)

    new_df = pd.DataFrame(new_df)
    new_df.to_csv(output_file, index=False)

def main():
    input_file = r"D:\water\Route 35 csv- July Cons.csv"
    output_file = input_file.split('.')[0] + '_updated.csv'
    process_csv(input_file, output_file)

if __name__ == '__main__':
    main()
